Git
git remote add origin https://github.com/duggurd/mini_prosjekt_ml
git add --all
git commit -m "2"
git push --all

# IMPORTS

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np


RANDOM_STATE = 420

# Load Data

from google.colab import drive

drive.mount('/content/drive')

In [ ]:
path = "G:\My Drive\AW_Academy\mini_project_week6\competitive-data-science-predict-future-sales/"
path2 = 'data/'

sales = pd.read_csv(f"G:\My Drive\AW_Academy\mini_project_week6\competitive-data-science-predict-future-sales/sales_train.csv") 
item_cats = pd.read_csv(f"{path}item_categories.csv") 
items = pd.read_csv(f"{path}items.csv") 
shops = pd.read_csv(f"{path}shops.csv") 
test = pd.read_csv(f"{path}test.csv")

print(f"Train shape: {sales.shape} \nTest shape: {test.shape}")

# Data washing

In [ ]:
# Adding test salesset to training salessaet for easy transformation of both
test["date_block_num"] = 34

sales = pd.concat([sales, test.drop(columns=["ID"])]).fillna(0)

In [ ]:
# Removing prices and counts that make no sense
sales = sales[sales["item_price"] > 0]
sales = sales[sales["item_cnt_day"] >= 0]

# Converting to datetime
sales["date"] = pd.to_datetime(sales["date"], dayfirst=True)

# Adding dayofweek column
sales["dof"] = sales["date"].dt.dayofweek

# Adding day of year column
sales["doy"] = sales["date"].dt.day_of_year

# Adding day of month column
sales["dom"] = sales["date"].dt.day

# Creating revenue column
sales["revenue"] = sales["item_cnt_day"] * sales["item_price"]

test["date_block_num"] = 34

sales = pd.concat([sales, test.drop(columns=["ID"])]).fillna(0)

In [ ]:
# Creating broader categories and encoding
item_cats["group_name"] = item_cats["item_category_name"].str.extract(r"(^[\w\s]*)")
item_cats["group_name"] = item_cats["group_name"].str.strip()
le = LabelEncoder()
item_cats["group_name_id"] = le.fit_transform(item_cats["group_name"])

item_cats.sample(5)

In [ ]:
# Cleaning item_name
items["item_name"] = items["item_name"].str.lower()
items["item_name"] = items["item_name"].str.replace(".", "", regex=False)

# Extracting first n chars of item_name
items["item_name_no_space"] = items["item_name"].str.replace(" ", "", regex=False)
items["item_name_first_4"] = [x[:4] for x in items["item_name_no_space"]]
items["item_name_first_6"] = [x[:6] for x in items["item_name_no_space"]]
del items["item_name_no_space"]

# Label encoding first n chars
items["item_name_first_4"] = le.fit_transform(items["item_name_first_4"])
items["item_name_first_6"] = le.fit_transform(items["item_name_first_6"])

# Joining items to item_cats
items = items.merge(item_cats.set_index("item_category_id"), on="item_category_id")
items.sample(5)

In [ ]:
# Merging sales with items and categories
df = pd.merge(sales, items.drop(columns=["item_name", "item_category_name", "group_name"]), on=["item_id"])

In [ ]:
# Removing outliers
df = df[df["item_cnt_day"] < df["item_cnt_day"].quantile(0.99)]
df = df[df["item_price"] < df["item_price"].quantile(0.99)]

In [ ]:
# Remove date - since we have date_block
df.drop('date',axis=1,inplace=True)

# Export Data

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='df.csv')  

df.to_csv('df.zip', index=False,
          compression=compression_opts)  